# Ejecutar pipelines

In [ ]:
pip show azure-ai-ml

## Conectar a workspace

In [1]:
# conectar
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client = MLClient.from_config(credential=DefaultAzureCredential())

print(f"Conectado al Workspace: {ml_client.workspace_name}")

Found the config file in: /config.json


Conectado al Workspace: naturgyml4


## Cargar los componentes

In [3]:
from azure.ai.ml import load_component

ing_prep_data = ml_client.components.get(name="ing_prep_data_housing_ric", version=2)
train_model_housing = ml_client.components.get(name="train_model_housing_ric", version=2)
eval_model_housing = ml_client.components.get(name="eval_model_housing_ric", version=10)

## Build del pipeline

In [4]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.dsl import pipeline


@pipeline(name="house_pricing_pipeline", description="Pipeline to train a house pricing model")
def house_pricing(input_data, alpha):
    clean_data = ing_prep_data(input_data=input_data)
    train_model = train_model_housing(input_data=clean_data.outputs.output_data, alpha=alpha)
    eval_model = eval_model_housing(input_data=train_model.outputs.output_data, model_input=train_model.outputs.model_output)

    return {
        "pipeline_job_transformed_data": clean_data.outputs.output_data,
        "pipeline_job_trained_model": train_model.outputs.model_output,
        "pipeline_job_output_data": train_model.outputs.output_data,
    }

In [5]:
# definir job de pipeline
pipeline_job = house_pricing(Input(type=AssetTypes.URI_FILE, path="azureml:housing_prices:2"), alpha=0.5)

# change the output mode
pipeline_job.outputs.pipeline_job_transformed_data.mode = "upload"
pipeline_job.outputs.pipeline_job_trained_model.mode = "upload"

# set pipeline level compute
pipeline_job.settings.default_compute = "ricardoenm2"

# set pipeline level datastore
pipeline_job.settings.default_datastore = "training_data_ric"
pipeline_job.settings.force_rerun = True

In [6]:
print(pipeline_job)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


display_name: house_pricing_pipeline
description: Pipeline to train a house pricing model
type: pipeline
inputs:
  input_data:
    type: uri_file
    path: azureml:housing_prices:2
  alpha: 0.5
outputs:
  pipeline_job_transformed_data:
    mode: upload
    type: uri_folder
  pipeline_job_trained_model:
    mode: upload
    type: mlflow_model
  pipeline_job_output_data:
    type: uri_folder
jobs:
  clean_data:
    type: command
    inputs:
      input_data:
        path: ${{parent.inputs.input_data}}
    outputs:
      output_data: ${{parent.outputs.pipeline_job_transformed_data}}
    resources:
      instance_count: 1
    component:
      $schema: https://azuremlschemas.azureedge.net/latest/commandComponent.schema.json
      name: ing_prep_data_housing_ric
      version: '2'
      display_name: Ingesta y Preprocesamiento de Datos Housing
      type: command
      inputs:
        input_data:
          type: uri_file
          optional: false
      outputs:
        output_data:
         

## Enviar el job de pipeline

In [7]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job,
    experiment_name="pipeline_housing_prices_ric",
    display_name="housing_prices_pipeline_job_ric"
)

pipeline_job

pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.MLFlowModelJobOutput'> and will be ignored
pathOnCompute is not a known attribute of class <class 'azure.ai.ml._restclient.v2023_04_01_preview.models._models_py3.UriFolderJobOutput'> and will be ignored


Experiment,Name,Type,Status,Details Page
pipeline_housing_prices_ric,icy_kumquat_fqy4nkkf8z,pipeline,NotStarted,Link to Azure Machine Learning studio
